In [ ]:
print("hello")

In [ ]:
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
os.getenv("GOOGLE_API_KEY")
os.getenv("GROQ_API_KEY")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")

In [ ]:
from langchain_groq import ChatGroq

llm_groq = ChatGroq(model="llama-3.1-8b-instant", temperature=0, max_tokens=128)

llm_groq.invoke("Hello!")

In [ ]:
llm.invoke("Hello!")

## Building our First ReAct Agent

In [ ]:
from langchain.agents import create_agent

my_tools = []
my_agent = create_agent(model=llm_groq,tools=my_tools)


In [ ]:
my_agent

In [ ]:
my_agent.invoke({"messages":[("user","Hi there!")]})


## Adding Dummy Tool

In [ ]:
from langchain.tools import tool

In [ ]:
@tool("uppercase_tool", description="Converts input text to uppercase")
def dummy_tool(inputText:str)-> str:
    """Convert input text to uppercase. This is just for testing.
     Arguments:
        inputText {str} -- input text to be converted to uppercase
    """
    return f"Output : {inputText.upper()}"

In [ ]:
tools = [dummy_tool]

agent_with_dummy_tool = create_agent(model=llm_groq, tools=tools)

In [ ]:
agent_with_dummy_tool

In [ ]:
res = agent_with_dummy_tool.invoke({"messages":[("user","use the dummy tool on 'react agents'")]})

In [ ]:
res

In [ ]:
res2 = agent_with_dummy_tool.invoke({"messages":[("user","Hi there, how are you'")]})

In [ ]:
res2

In [ ]:
res3 = agent_with_dummy_tool.invoke({"messages":[("user","convert 'langchain' to uppercase")]})

In [ ]:
res3

## Adding Weather Tool

In [ ]:
import requests

In [ ]:
@tool
def get_weather(city: str)-> dict:
    """Get the current weather for a given city"""
    try:
        geo_url=f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1&language=en&format=json"
        geo_data = requests.get(geo_url).json()

        if not geo_data.get("results"):
            return {"error": f"City '{city}' not found"}
        
        location = geo_data["results"][0]
        lat, lon = location["latitude"], location['longitude']

        weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,weather_code"
        date = requests.get(weather_url).json()

        temp = date["current"]["temperature_2m"]
        return {"city":city, "temp":temp, "unit":"C"}





    except Exception as e:
        return {"error":str(e)}


In [ ]:
print(get_weather.invoke("Dubai"))

In [ ]:
tools_with_weather = [dummy_tool, get_weather]

weather_agent = create_agent(model=llm_groq, tools=tools_with_weather)

In [ ]:
weather_agent

In [ ]:
response1= weather_agent.invoke({"messages":[("user","Can you give me the current temp of bhopal")]})

In [ ]:
response1

In [ ]:
response2= weather_agent.invoke({"messages":[("user","Can you convert 'Hello there' in uppercase and also give me the temp of mumbai")]})

In [ ]:
response2